<a href="https://colab.research.google.com/github/seyf1elislam/LocalLLM_OneClick_Colab/blob/main/Run_GPTQ_and_fullprecision_LLM_models_in_TextGen_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $\color{#19ABEA}{\text{Run GPTQ and full precision LLM models in Latest Version TextGen-webui :}}$


this [Notebook](https://github.com/seyf1elislam/LocalLLM_OneClick_Colab) allow u run  any quantized GPTQ file from any Hugging face repository as long as it fits the colab Vram and ram.

>follow this [githubRepository](https://github.com/seyf1elislam/LocalLLM_OneClick_Colab) to get the updates of this noteboook.

> Created by :
> 😇 github :   [@seyf1elislam](https://github.com/seyf1elislam)
> 🤗 Huggingface : [@seyf1elislam](https://huggingface.co/seyf1elislam)

## where can i find GPTQ quantized models ?
> you can check those 🤗 huggingface repos :
- [Thebloke](https://huggingface.co/TheBloke) #1 best GPTQ source ,contains hundreds of quantized models.
- [using search](https://huggingface.co/models?sort=trending&search=GPTQ) here u can find all GPTQ files on  huggingface.

## need good models to try ?
> you can try those :  
- [TheBloke/SOLAR-10.7B-v1.0-GPTQ](https://huggingface.co/TheBloke/SOLAR-10.7B-v1.0-GPTQ)
- [TheBloke/Kunoichi-7B-GPTQ](https://huggingface.co/TheBloke/Kunoichi-7B-GPTQ)
- [TheBloke/Starling-LM-7B-alpha-GPTQ](https://huggingface.co/TheBloke/Starling-LM-7B-alpha-GPTQ)

## some Tips ?
> most mistral models goes with 8k context length if u want to use longer context u need to make sure the models support longer context.

> if u want to run model higher then 13B (such as 20B,4x7b..) on colab u may need to reduce the offloaded gpu models to split the ram usage between gpu vram and system ram. (slower but it works 😉)


In [ ]:
#@title # $\color{#19ABEA}{\text{Download and install requirements}}$ {display-mode: "form"}
#@markdown $\color{#19ABEA}{\text{run this only once at the start}}$

## @markdown `Note :no need to check these checkboxes unless u facing a problem`
#@markdown ---
#@markdown `check this if u need to change the version only`
delete_existing_texgen_webui_folder =False #@param {type:"boolean"}
if delete_existing_texgen_webui_folder :
  !rm -rf /content/text-generation-webui
#@markdown ---
#@markdown ### TextGen webui version :
#@markdown `note : gemma models are supported in snapshot-2024-02-25 or newer`
%cd /content
branch = "dev" # @param ["dev", "snapshot-2024-01-07", "snapshot-2024-01-14", "snapshot-2024-01-28", "snapshot-2024-02-11", "snapshot-2024-02-25", "snapshot-2024-03-03", "snapshot-2024-03-10", "snapshot-2024-03-17", "snapshot-2024-03-24", "snapshot-2024-03-31", "snapshot-2024-04-07", "snapshot-2024-04-14", "snapshot-2024-04-21", "snapshot-2024-04-28"]
branch = f"-b {branch}"
#@markdown `to download the latest version of TextGen Webui use use_latest ,this will ignore the branch name.`
use_latest =False #@param {type:"boolean"}

if use_latest :
  branch = ""
!git clone {branch} https://github.com/oobabooga/text-generation-webui

%cd /content/text-generation-webui
!pip install -r requirements.txt
!pip install git+https://github.com/UWUplus/flask-cloudflared
!pip install -r extensions/openai/requirements.txt --upgrade
!apt-get -y install -qq aria2
#@markdown ----
reinstall_llama_cpp_python = False # @param {type:"boolean"}
if reinstall_llama_cpp_python :
  !pip uninstall -y llama-cpp-python
  !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir


reinstall_flash_attention = False # @param {type:"boolean"}
if reinstall_flash_attention:
  !pip uninstall -y flash-attn
  !pip install --no-build-isolation flash-attn==2.3.0
#@markdown ----

from IPython.display import clear_output
clear_output()
print("Done")

In [ ]:
#@title #  $\color{#19ABEA}{Download Model}$ {display-mode: "form"}
#@markdown ## $\color{#19ABEA}{\text{Copy the RepoName here }}$
#@markdown you can use GPTQ repository model here such as
#@markdown   :`TheBloke/mistral-ft-optimized-1218-GPTQ`
#@markdown  or :`TheBloke/xDAN-L1-Chat-RL-v1-GPTQ` etc...
repo_name = "TheBloke/mistral-ft-optimized-1218-GPTQ" # @param ["TheBloke/Mistral-7B-OpenOrca-GPTQ", "TheBloke/zephyr-7B-alpha-GPTQ", "TheBloke/zephyr-7B-beta-GPTQ", "TheBloke/neural-chat-7B-v3-1-GPTQ", "TheBloke/mistral-ft-optimized-1218-GPTQ", "TheBloke/xDAN-L1-Chat-RL-v1-GPTQ", "TheBloke/openchat-3.5-1210-starling-slerp-GPTQ", "TheBloke/Loyal-Macaroni-Maid-7B-GPTQ", "TheBloke/Valkyrie-V1-GPTQ", "TheBloke/Sonya-7B-GPTQ", "TheBloke/Starling-LM-7B-alpha-GPTQ", "TheBloke/SOLAR-10.7B-v1.0-GPTQ"] {allow-input: true}
# repo_name = "xDAN-AI/xDAN-L1-Chat-RL-v1" # @param ["teknium/OpenHermes-2.5-Mistral-7B", "xDAN-AI/xDAN-L1-Chat-RL-v1"]  { allow-input: true}

#@markdown  in colab gpu (15Gvram) you can use :
#@markdown - `13b model GPTQ `
#@markdown - `7b model GPTQ`
#@markdown - you can use full precision 13b , 7b modeles but using flags like `--load-in-4bit` or `--load-in-8bit`  its a bit slower then quantized versions but works well

repo_link =f"https://huggingface.co/{repo_name}"
print(repo_link)
%cd /content/text-generation-webui/models
!git lfs install
!git clone $repo_link
print("Done")

In [ ]:
#@title ##  $\color{#19ABEA}{RunOnly}$ {display-mode: "form"}
# %cd /content/text-generation-webui
#@markdown  $\color{#19ABEA}{\text{Enter ur Context size or let it Unckechet to use the default size}}$
params =" "
#@markdown _____
api =True # @param {type:"boolean"}
if api :params +=" --api --public-api --listen "
api_only_mode =False # @param {type:"boolean"}
if api_only_mode : params+=" --nowebui "
#@markdown _____
use_custom_context_len = False # @param {type:"boolean"}
context_len=8192 #@param
if use_custom_context_len : params += f" --n_ctx  {context_len} "
#@markdown _____
use_load_in_flag = False # @param {type:"boolean"}
load_in = "--load-in-8bit" # @param ["--load-in-8bit","--load-in-4bit"] {type:"string"}
if use_load_in_flag: params +=f" {load_in} "
#@markdown _____
use_custom_params = False # @param {type:"boolean"}
custom_params = "" # @param {type:"string"}
if use_custom_params : params +=custom_params


#@markdown _____
# params
%cd /content/text-generation-webui
model_file_name =repo_name.split('/')[-1]
!python server.py   --share  --n-gpu-layers 100000  	--model $model_file_name   $params